In [83]:
import collections
from collections import Counter

#import helper
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline



import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split

#from tabulate import tabulate

import gc
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

In [84]:
import joblib

In [85]:
# loading
with open('tokenizerc.pickle', 'rb') as handle:
    tknizer_c = pickle.load(handle)
    # loading
with open('tokenizeruc.pickle', 'rb') as handle:
    tknizer_uc = pickle.load(handle)

In [86]:
vocab_size_uc=len(tknizer_uc.word_index.keys())
print(vocab_size_uc)
vocab_size_c=len(tknizer_c.word_index.keys())
print(vocab_size_c)

3008
7500


In [87]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

      super().__init__()
      self.inp_vocab_size= inp_vocab_size
      self.embedding_size = embedding_size
      self.input_length = input_length
      self.lstm_size = lstm_size
      self.lstm_output = 0
      self.lstm_state_h = 0
      self.lstm_state_c = 0
      self.embedding = Embedding(input_dim=self.inp_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                            mask_zero=True, name="embedding_layer_encoder")
      self.lstm = LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,states):
      input_embedd= self.embedding(input_sequence)
      self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
      return self.lstm_output, self.lstm_state_h,self.lstm_state_c


    
    def initialize_states(self,batch_size):
      #check
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      initial_hidden_state =tf.zeros(shape=[batch_size,self.lstm_size],dtype=tf.int32)
      initial_cell_state =tf.zeros(shape=[batch_size,self.lstm_size],dtype=tf.int32)
      return initial_hidden_state,initial_cell_state

In [88]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):
    super().__init__()
    self.scoring_function = scoring_function
    self.att_units = att_units

    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      # encoder output
      self.Dot = tf.keras.layers.Dot(axes=(1, 2))
    if scoring_function == 'general':
      self.W1= tf.keras.layers.Dense(att_units)
      self.Dot = tf.keras.layers.Dot(axes=(1, 2))
    elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
        self.W2 = tf.keras.layers.Dense(att_units)
        self.W3 = tf.keras.layers.Dense(att_units)
        self.V = tf.keras.layers.Dense(1)
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state,1)
        dhs = tf.transpose(decoder_hidden_state,(0,2,1))
        #print(dhs.shape)
        #print(encoder_output.shape)
        x = self.Dot([dhs,encoder_output])
        a = tf.nn.softmax(tf.transpose(x,(0,2,1)),axis=1)
        c = a*encoder_output
        c = tf.reduce_sum(c,axis = 1)
        #print(c.shape,a.shape)
        return c,a




  


    elif self.scoring_function == 'general':
        # Implement General score function here
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state,1)
        wa = self.W1(encoder_output)
        dhs = tf.transpose(decoder_hidden_state,(0,2,1))
        x = self.Dot([dhs,wa])
        
        a = tf.nn.softmax(tf.transpose(x,(0,2,1)),axis=1)
        c = a*encoder_output
        c = tf.reduce_sum(c,axis = 1)
        #print(c.shape,a.shape)
        return c,a

    elif self.scoring_function == 'concat':
        # Implement General score function here
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state,1)
        e = self.V(tf.nn.tanh(self.W2(decoder_hidden_state)+self.W3(encoder_output)))
        a = tf.nn.softmax(e,axis = 1)
        c = a*encoder_output
        c = tf.reduce_sum(c,axis = 1)
        m = self.W2
        #print(c.shape,a.shape)
        #print(m.shape)
        return c,a

In [89]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
    super().__init__()
    self.tar_vocab_size = tar_vocab_size 
    self.embedding_dim = embedding_dim
    self.input_length =  input_length
    self.dec_units = dec_units
    self.score_fun = score_fun
    self.att_units = att_units
# Initialize decoder embedding layer, LSTM and any other objects needed
    self.lstm_output = 0
    self.lstm_state_h = 0
    self.lstm_state_c = 0
    self.embedding = Embedding(input_dim=self.tar_vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                            mask_zero=True, name="embedding_layer_encoder")
    self.lstm = LSTM(att_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
    self.dense   = Dense(self.tar_vocab_size)
    self.attention = Attention(scoring_function = self.score_fun,att_units = self.att_units)




  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    
    input_embedding= self.embedding(input_to_decoder)
    context,similarities = self.attention(state_h,encoder_output)
    #print(input_embedding.shape)
    #print(context.shape)
    concat = tf.keras.layers.Concatenate(axis = 2)([input_embedding,tf.expand_dims(context,1)])
    decoder_output,*states = self.lstm(concat)
    
    #print(states[0].shape)
    decoder_hidden,decoder_cell = states[0],states[1]

    output =self.dense(decoder_output[:,:,:self.dec_units])
    #print(decoder_hidden.shape)
    return tf.squeeze(output,axis = 1),decoder_hidden,decoder_cell,similarities,context
#context_vector = 32116
#i2d_output = 32112

In [90]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
                
      super().__init__()
      #Intialize necessary variables and create an object from the class onestepdecoder
      self.out_vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim

      self.input_length = input_length
      self.dec_units = dec_units
      self.score_fun = score_fun
      self.att_units = att_units
      self.onestepdecoder=One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)  

        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        all_outputs = tf.TensorArray(tf.float32, size=40, name = 'output_arrays',dynamic_size = True,clear_after_read = False)
        for i in range(40):
          output,decoder_hidden_state ,decoder_cell_state,attention_weights,context_vector = self.onestepdecoder(input_to_decoder[:,i:i+1],encoder_output,decoder_hidden_state,decoder_cell_state)#sequence is correct
          all_outputs = all_outputs.write(i,output)

       # all_outputs.mark_used()


        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])
        return all_outputs

In [91]:
class encoder_decoder(tf.keras.Model):
    def __init__(self,encoder_inputs_length,decoder_inputs_length,output_vocab_size,batch_size):
    #Intialize objects from encoder decoder
        super().__init__()
        self.encoder = Encoder(inp_vocab_size = vocab_size_c+2,embedding_size = 100,lstm_size = 512,input_length = encoder_inputs_length)
        self.decoder = Decoder(out_vocab_size = vocab_size_uc+2,embedding_dim = 100,input_length = decoder_inputs_length ,dec_units = 512,score_fun = 'concat',att_units = 512)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        self.batch_size = batch_size
        self.embedding = Embedding(input_dim=vocab_size_c+1, output_dim=100, input_length=40,
                               mask_zero=True, name="embedding_layer_encoder")

  
    def call(self,data):
        #Intialize encoder states, Pass the encoder_sequence to the embedding layer
        # Decoder initial states are encoder final states, Initialize it accordingly
        # Pass the decoder sequence,encoder_output,decoder states to Decoder
        # return the decoder output
        input,output = data[0],data[1]
        initial_state = self.encoder.initialize_states(self.batch_size)
        input_embedd= input
        encoder_output,encoder_h,encoder_c = self.encoder(input_embedd,initial_state)
        decoder_output = self.decoder(output,encoder_output,encoder_h,encoder_c)
        return decoder_output

In [92]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [93]:
model  = encoder_decoder(encoder_inputs_length=40,decoder_inputs_length=40,output_vocab_size=vocab_size_uc,batch_size =1)


In [94]:
model.built = True

In [95]:
model.load_weights('my_model2')

In [96]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         Save the attention weights
         And get the word using the tokenizer(word index) and then store it in a string.
  E. Call plot_attention(#params)
  F. Return the predicted sentence
  '''
  attention_plot = np.zeros((40,40))
  input_sequences = tknizer_c.texts_to_sequences([input_sentence])
  encoder_sequence = pad_sequences(input_sequences, maxlen = 40, dtype='int32', padding='post')
  encoder_output,state_h,state_c= model.layers[0](encoder_sequence,model.layers[0].initialize_states(1))
  dec_input = tknizer_uc.word_index['<start>']
  dec_input = (np.array(dec_input)).reshape(1,1)
  states = [state_h,state_c]
  j = 0
  final = []
  while j!=40:
    predicted_out,state_h,state_c,attention_weights,context_vector=model.layers[1].onestepdecoder(dec_input,encoder_output,state_h,state_c)
    #att_w.append(state_c)
    
    attention_weights = tf.reshape(attention_weights,(-1,))
    attention_plot[j] = attention_weights.numpy()
    #______________________________________

    output = predicted_out
    states = [state_h,state_c] #automatically get updated as above
    output= tf.argmax(output[0]).numpy()
    #print(output)
    b = get_key(output)
    if b == '<end>':
      break
    else:
      final.append(get_key(output))
      dec_input =tf.expand_dims([output], 0)
      j+=1
 # plot_attention(att_w,input_sentence,final)
  
  return ' '.join(final),input_sentence,attention_plot

In [97]:
def get_key(val):
    for key, value in tknizer_uc.word_index.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [98]:
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

def preprocess(text):
    # convert all the text into lower letters
    # use this function to remove the contractions: https://gist.github.com/anandborad/d410a49a493b56dace4f814ab5325bbd
    # remove all the spacial characters: except space ' '
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    return text

def preprocess_c(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    # we have found these characters after observing the data points, feel free to explore more and see if you can do find more
    # you are free to do more proprocessing
    # note that the model will learn better with better preprocessed data 
    
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


In [99]:
def Final(x):
  z = []
  dfx  = pd.read_csv(x)
  dfx = dfx.drop("Unnamed: 0",axis = 1)
  dfx = dfx.drop("Unnamed: 0.1",axis = 1)
 
  c  = dfx.columns[0]
  dfx["c_l"] =  dfx[c].str.split().apply(len)
  dfx = dfx[dfx['c_l'] < 40]
  dfx = dfx.drop(['c_l'],axis = 1)
  dfx[c] = dfx[c].apply(preprocess_c)
  for j in range(len(dfx)):
        f,i,at  =  predict(dfx.iloc[j][0])
        z.append(f)
  return z
  

In [100]:
import numpy as np

In [ ]:
# Importing necessary modules
from flask import Flask, render_template, request
import pickle
import numpy as np
# Loading the Models


# Instantiate app object 
app = Flask(__name__,template_folder='templates')
# Setting up app routes
@app.route('/')
def man(): 
    return render_template('Home.html')
@app.route('/home', methods=['POST'])
def home():
    feature1 =(request.form['filename'])
    predicted_values = Final(feature1)
    return render_template('Home.html', prediction_text= (predicted_values))
# Flask server
if __name__=="__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Feb/2022 11:01:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2022 11:01:51] "POST /home HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2022 11:04:12] "POST /home HTTP/1.1" 500 -
